# Portfolio Data Collection from Interactive Brokers

This notebook demonstrates how to extract portfolio holdings from Interactive Brokers (IB) Gateway and store them in a database for further analysis.

## Overview

This notebook covers:
- Connecting to Interactive Brokers Gateway
- Extracting account summary and portfolio positions
- Processing and transforming the data
- Storing portfolio holdings in a database

## Prerequisites

Before running this notebook, ensure you have:
- Interactive Brokers Gateway running on localhost:4001
- Required Python packages installed
- Database credentials stored in keyring
- Appropriate database schema and functions

## Required Libraries

In [13]:
import sys
import os

# Get the current notebook's directory and go up to parent
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

print(f"Current directory: {current_dir}")
print(f"Added to path: {parent_dir}")

# Verify the data_engineering folder exists
data_eng_path = os.path.join(parent_dir, 'data_engineering')
print(f"data_engineering exists: {os.path.exists(data_eng_path)}")

if os.path.exists(data_eng_path):
    print(f"data_engineering contents: {os.listdir(data_eng_path)}")

Current directory: c:\Users\menon\OneDrive\Documents\SourceCode\InvestmentManagement\scripts
Added to path: c:\Users\menon\OneDrive\Documents\SourceCode\InvestmentManagement
data_engineering exists: True
data_engineering contents: ['database', 'eod_data', 'fundamental_data', '__init__.py', '__pycache__']


In [14]:
from datetime import datetime, timedelta
import keyring
import pandas as pd
from ib_insync import *
from data_engineering.database import db_functions as database
import nest_asyncio
from ib_insync import *

## Utility Functions

### Get Trading Day Function

This function ensures we always work with valid trading days (excluding weekends):

In [15]:
def get_trading_day():
    """
    Get the most recent trading day (excluding weekends).
    
    Returns:
        str: Date in 'YYYY-MM-DD' format
    """
    today = datetime.today()
    while today.weekday() >= 5:  # 5 = Saturday, 6 = Sunday
        today -= timedelta(days=1)
    return today.strftime('%Y-%m-%d')

# Test the function
print(f"Current trading day: {get_trading_day()}")

Current trading day: 2025-08-06


## Interactive Brokers Connection

### Establishing Connection

**Note**: Ensure your IB Gateway is running and configured to accept API connections on port 4001.

In [16]:
# Apply the patch for Jupyter compatibility
nest_asyncio.apply()

# Check if connection exists and close it
try:
    if 'ib' in globals() and ib.isConnected():
        print("Existing connection found. Disconnecting...")
        ib.disconnect()
        print("Disconnected successfully")
except NameError:
    print("No existing connection variable found")
except Exception as e:
    print(f"Error checking/closing existing connection: {e}")

# Connect to IB Gateway
ib = IB()

try:
    print("Attempting to connect to IB Gateway...")
    ib.connect('127.0.0.1', 4001, clientId=1, timeout=10)
    print(f"Connected to IB Gateway: {ib.isConnected()}")
    
    # Test with a simple request
    if ib.isConnected():
        print("Connection successful!")
        # Get account summary
        account_summary = ib.accountSummary()
        print(f"Retrieved {len(account_summary)} account summary items")
    else:
        print("Connection failed")
        
except Exception as e:
    print(f"Connection error: {e}")
    print("Make sure IB Gateway is running and API is enabled")

Attempting to connect to IB Gateway...
Connected to IB Gateway: True
Connection successful!
Retrieved 119 account summary items


### Account Summary Extraction

In [17]:
# Get account summary
account_summary = ib.accountSummary('U20761295')
account_summary_df = pd.DataFrame([
    {'tag': row.tag, 'value': row.value} 
    for row in account_summary
])

print("Account Summary:")
print(account_summary_df.head())

Account Summary:
                   tag                      value
0          AccountType  IRA-UK RETIREMENT ACCOUNT
1              Cushion                          1
2  LookAheadNextChange                          0
3          AccruedCash                       0.00
4       AvailableFunds                       3.90


## Portfolio Holdings Extraction

### Getting Current Positions

In [18]:
# Get portfolio holdings
positions = ib.positions()
portfolio_holdings = []

for pos in positions:
    contract = pos.contract
    portfolio_holdings.append({
        'as_of_date': get_trading_day(),
        'portfolio_short_name': pos.account,
        'symbol': contract.symbol,
        'security_type': contract.secType,
        'exchange': contract.exchange,
        'currency': contract.currency,
        'held_shares': pos.position,
        'avg_cost': pos.avgCost,
    })

df_portfolio_holdings_data = pd.DataFrame(portfolio_holdings)

print(f"Retrieved {len(df_portfolio_holdings_data)} portfolio positions:")
print(df_portfolio_holdings_data.head())

Retrieved 5 portfolio positions:
   as_of_date portfolio_short_name symbol security_type exchange currency  \
0  2025-08-06            U20758256   MSCI           STK     NYSE      USD   
1  2025-08-06            U20761295     MU           STK   NASDAQ      USD   
2  2025-08-06            U20761295   EMBC           STK   NASDAQ      USD   
3  2025-08-06            U20761295   IONQ           STK     NYSE      USD   
4  2025-08-06            U20761295   UBER           STK     NYSE      USD   

   held_shares   avg_cost  
0          1.0  533.20000  
1          1.0  124.86930  
2          1.0   10.85750  
3          2.0   40.97410  
4          2.0   93.00905  


### Data Structure Overview

The portfolio holdings DataFrame contains:
- **as_of_date**: The trading date for the data
- **portfolio_short_name**: Account identifier
- **symbol**: Security symbol
- **security_type**: Type of security (STK, OPT, etc.)
- **exchange**: Exchange where the security is traded
- **currency**: Currency denomination
- **held_shares**: Number of shares held
- **avg_cost**: Average cost per share

## Database Operations

### Secure Credential Management

In [19]:
# Secure credentials using keyring
service_name = "ihub_sql_connection"
db = keyring.get_password(service_name, "db")
db_user = keyring.get_password(service_name, "uid")
db_password = keyring.get_password(service_name, "pwd")

print("Database credentials retrieved from keyring")

Database credentials retrieved from keyring


### Database Connection

In [20]:
# Establish database connection
engine, connection, session = database.get_db_connection()
print("Database connection established")

Database connection successful.
Database connection established


### Data Enrichment

In [21]:
# Read reference data from database
df_securities = database.read_security_master(session, engine)
df_portfolio_data = database.read_portfolio(
    session, 
    engine, 
    df_portfolio_holdings_data['portfolio_short_name'].unique().tolist()
)

print(f"Securities master data: {len(df_securities)} records")
print(f"Portfolio reference data: {len(df_portfolio_data)} records")

Securities master data: 159117 records
Portfolio reference data: 1 records


### Data Merging and Transformation

In [22]:
# Merge portfolio holdings with reference data
df_portfolio_market_data = pd.merge(
    pd.merge(df_portfolio_data, df_portfolio_holdings_data),
    df_securities, 
    on='symbol', 
    how='inner'
)

# Select final columns for database storage
df_portfolio_market_data = df_portfolio_market_data[[
    'as_of_date',
    'port_id', 
    'security_id', 
    'held_shares'
]]

print("Final portfolio data structure:")
print(df_portfolio_market_data.head())
print(f"Total records to be stored: {len(df_portfolio_market_data)}")

Final portfolio data structure:
   as_of_date  port_id  security_id  held_shares
0  2025-08-06        1        99102          1.0
1  2025-08-06        1        61581          1.0
2  2025-08-06        1        80174          2.0
3  2025-08-06        1       135038          2.0
Total records to be stored: 4


### Data Storage

In [23]:
# Write portfolio holdings to database
database.write_portfolio_holdings(df_portfolio_market_data, session)
print("Portfolio holdings successfully written to database")

Portfolio holdings successfully written to database


## Cleanup and Disconnection

In [24]:
# Disconnect from IB Gateway
ib.disconnect()
print("Disconnected from IB Gateway")

Disconnected from IB Gateway


## Summary

This notebook demonstrates a complete workflow for:

1. **Data Extraction**: Connecting to Interactive Brokers Gateway and extracting portfolio positions
2. **Data Transformation**: Converting raw position data into a structured format
3. **Data Enrichment**: Merging with reference data from the database
4. **Data Storage**: Persisting the processed data for analysis


### Next Steps

Consider enhancing this workflow with:
- Error handling and logging
- Data validation checks
- Automated scheduling


### Troubleshooting

**Common Issues:**
- **Connection Failed**: Ensure IB Gateway is running and API is enabled
- **Authentication Error**: Verify keyring credentials are correctly stored
- **Data Mismatch**: Check that symbols in positions match security master data
- **Database Error**: Confirm database schema and permissions are correct